In [158]:
import pandas as pd
import numpy as np
import calendar
import time
from time import strptime
from datetime import datetime
import requests
from matplotlib import pyplot as plt
import random

from toby_config import toby_api_key
from pprint import pprint

In [44]:
# Installing bokeh in order to plot on top of a map

import bokeh.io
from bokeh.models import (
  GMapPlot, GMapOptions, ColumnDataSource, Circle, LogColorMapper, BasicTicker, ColorBar,
    DataRange1d, PanTool, WheelZoomTool, Range1d, Annotation, Legend, LegendItem, GlyphRenderer)
    
from bokeh.plotting import figure, show, output_file, output_notebook

In [3]:
df= pd.read_csv("crime.csv")
df['semi_year'] = df['REPORTED_DATE'].map(lambda x: x)
df['REPORTED_DATE'] = pd.to_datetime(df['REPORTED_DATE'])
df['year'] = df['REPORTED_DATE'].map(lambda x: str(x.year) +'A' if x.month < 7 else str(x.year) +"B")
df= df[df['IS_TRAFFIC'] == 0].copy()
df.head()

,INCIDENT_ID,OFFENSE_ID,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,REPORTED_DATE,INCIDENT_ADDRESS,...,GEO_LON,GEO_LAT,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,IS_CRIME,IS_TRAFFIC,Year,semi_year,year
0,2.016377e+09,2.020000e+15,5213,0,weapon-unlawful-discharge-of,all-other-crimes,6/15/2016 23:31,NaN,2016-06-15,NaN,...,-104.809881,39.773188,5,521,montbello,1,0,2016,6/15/2016,2016A
1,2.018600e+10,2.020000e+16,2399,0,theft-other,larceny,10/11/2017 12:30,10/11/2017 16:55,2018-01-29,NaN,...,-104.781434,39.785649,5,522,gateway-green-valley-ranch,1,0,2018,1/29/2018,2018A
2,2.016600e+10,2.020000e+16,2305,0,theft-items-from-vehicle,theft-from-motor-vehicle,3/4/2016 20:00,4/25/2016 8:00,2016-04-26,2932 S JOSEPHINE ST,...,-104.957381,39.663490,3,314,wellshire,1,0,2016,4/26/2016,2016A
3,2.018723e+08,2.020000e+14,2399,0,theft-other,larceny,1/30/2018 19:20,NaN,2018-01-30,705 S COLORADO BLVD,...,-104.941440,39.702698,3,312,belcaro,1,0,2018,1/30/2018,2018A
4,2.017411e+09,2.020000e+15,2303,0,theft-shoplift,larceny,6/22/2017 20:53,NaN,2017-06-23,2810 E 1ST AVE,...,-104.955370,39.717107,3,311,cherry-creek,1,0,2017,6/23/2017,2017A


In [131]:
def get_crime_type_locations(crime_type, year): #this function sorts data by crime type and year, outputs lat & lng
    year_filter = df[df['year'] == year].copy()
    crime_filter = year_filter[year_filter['OFFENSE_CATEGORY_ID'] == crime_type].copy()
    
    lat = crime_filter['GEO_LAT'].dropna()
    lat = lat[lat > 38].copy()
    lat = lat[lat < 41].copy()

    lng = crime_filter['GEO_LON'].dropna()
    lng = lng[lng > -105.5].copy()
    lng = lng[lng < -104.5].copy()
    return lat, lng

In [132]:
# plot is our figure, this allows us to plot multiple datasets on same map
def map_plot(plot, lat_data, lng_data, color_in, size_input): 
    color_list = [color_in for each in lat_data]
    source = ColumnDataSource(
        data=dict(
            lat= lat_data,
            lon= lng_data,
            size= size_input*np.ones(len(lat_data)),
            color= color_list
        )
    )
    circle = Circle(x="lon", y="lat", size="size", fill_color='color', fill_alpha=0.5, line_color=None)
    plot.add_glyph(source, circle)


In [133]:
import gmaps
gmaps.configure(api_key= toby_api_key)

In [156]:
df_mj_business= pd.read_csv("csv files//marijuana_active_business_licenses.csv")

df_mj_business['Clean_adress'] = df_mj_business['Facility Street Number'].astype(str)[:-2] + " "+ df_mj_business['Facility Street Name'] +" "+ df_mj_business['Facility Street Type']

df_mj_business['geocode']=df_mj_business['Facility Street Number']
df_mj_business.head()

,Business File Number,License Type,Entity Name,Trade Name,Current License Status,Expiration Date,Facility Street Number,Facility Pre-Direction,Facility Street Name,Facility Street Type,Facility Unit Number,Facility City,Facility Zip Code,Clean_adress,geocode
0,2015-BFN-0003448,Med Marijuana Inf Prod Manuf,DARK HORSE GENETICS LLC,NaN,License Issued - Active,9/14/2018 12:00:00 AM,2145.0,S,KALAMATH,ST,NaN,DENVER,80223.0,2145.0 KALAMATH ST,2145.0
1,2015-BFN-0003449,Med Marijuana Opt Prem Cultiv,DARK HORSE GENETICS LLC,NaN,License Issued - Active,9/14/2018 12:00:00 AM,2145.0,S,KALAMATH,ST,NaN,DENVER,80223.0,2145.0 KALAMATH ST,2145.0
2,2015-BFN-0003648,Med Marijuana Opt Prem Cultiv,SUMMIT WELLNESS,DEN-REC DENVER RECRATIONAL DISPENSARY,License Issued - Active,10/27/2018 12:00:00 AM,1901.0,S,BANNOCK,ST,NaN,DENVER,80223.0,1901.0 BANNOCK ST,1901.0
3,2015-BFN-0004345,Retail Marij Opt. Prem. Cultiv,M.J.L. Q LLC,WHOLE MEDS,License Issued - Active,8/27/2018 12:00:00 AM,5959.0,E,39TH,AVE,101,DENVER,80207.0,5959.0 39TH AVE,5959.0
4,2015-BFN-0004100,Medical Marijuana Center,TRILL EVOLUTIONS LLC,TRILL EVOLUTIONS,License Issued - Active,6/10/2019 12:00:00 AM,5550.0,N,JOLIET,ST,NaN,DENVER,80239.0,5550.0 JOLIET ST,5550.0


In [174]:
address_list = []
for each in range(len(df_mj_business)):
    address_list.append(str(df_mj_business.iloc[each]['Clean_adress']) + " " + str(df_mj_business.iloc[each]['Facility City']) + ", CO " + str(df_mj_business.iloc[each]['Facility Zip Code']))
    

In [191]:

geocode_url = "https://maps.googleapis.com/maps/api/geocode/json?address={}".format(address_list[478])
geocode_url = geocode_url + "&key={}".format(toby_api_key)
results = requests.get(geocode_url)
results_json = results.json()
pprint(results_json.get('status'))
#business_lats.append(results_json.get('results')[0].get('geometry').get('location').get('lat'))
#business_lngs.append(results_json.get('results')[0].get('geometry').get('location').get('lng'))

'OK'


In [192]:
business_lats = []
business_lngs = []
for address in address_list:
    try:
        geocode_url = "https://maps.googleapis.com/maps/api/geocode/json?address={}".format(address)
        geocode_url = geocode_url + "&key={}".format(toby_api_key)
        time.sleep(0.5)
        results = requests.get(geocode_url)
        results_json = results.json()
        if results_json.get('status') == 'OK':
            business_lats.append(results_json.get('results')[0].get('geometry').get('location').get('lat'))
            business_lngs.append(results_json.get('results')[0].get('geometry').get('location').get('lng'))
        else:
            continue
    except ValueError:
        continue

In [193]:
len(business_lats)

1109

In [222]:
df.head(30)

,INCIDENT_ID,OFFENSE_ID,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,REPORTED_DATE,INCIDENT_ADDRESS,...,GEO_LON,GEO_LAT,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,IS_CRIME,IS_TRAFFIC,Year,semi_year,year
0,2.016377e+09,2.020000e+15,5213,0,weapon-unlawful-discharge-of,all-other-crimes,6/15/2016 23:31,NaN,2016-06-15,NaN,...,-104.809881,39.773188,5,521,montbello,1,0,2016,6/15/2016,2016A
1,2.018600e+10,2.020000e+16,2399,0,theft-other,larceny,10/11/2017 12:30,10/11/2017 16:55,2018-01-29,NaN,...,-104.781434,39.785649,5,522,gateway-green-valley-ranch,1,0,2018,1/29/2018,2018A
2,2.016600e+10,2.020000e+16,2305,0,theft-items-from-vehicle,theft-from-motor-vehicle,3/4/2016 20:00,4/25/2016 8:00,2016-04-26,2932 S JOSEPHINE ST,...,-104.957381,39.663490,3,314,wellshire,1,0,2016,4/26/2016,2016A
3,2.018723e+08,2.020000e+14,2399,0,theft-other,larceny,1/30/2018 19:20,NaN,2018-01-30,705 S COLORADO BLVD,...,-104.941440,39.702698,3,312,belcaro,1,0,2018,1/30/2018,2018A
4,2.017411e+09,2.020000e+15,2303,0,theft-shoplift,larceny,6/22/2017 20:53,NaN,2017-06-23,2810 E 1ST AVE,...,-104.955370,39.717107,3,311,cherry-creek,1,0,2017,6/23/2017,2017A
5,2.018728e+08,2.020000e+14,5499,0,traf-other,all-other-crimes,1/31/2018 0:44,NaN,2018-01-31,2100 BLOCK E 17TH AVE,...,-104.961928,39.743149,6,622,city-park-west,1,0,2018,1/31/2018,2018A
6,2.018600e+10,2.020000e+16,2304,0,theft-parts-from-vehicle,theft-from-motor-vehicle,6/1/2017 12:15,1/26/2018 12:15,2018-01-26,995 N FEDERAL BLVD,...,-105.025543,39.732790,1,122,villa-park,1,0,2018,1/26/2018,2018A
7,2.018706e+08,2.020000e+14,5707,0,criminal-trespassing,all-other-crimes,1/30/2018 7:40,NaN,2018-01-30,E SPEER BLVD / N GRANT ST,...,-104.983794,39.723423,3,311,speer,1,0,2018,1/30/2018,2018A
9,2.018600e+10,2.020000e+16,2305,0,theft-items-from-vehicle,theft-from-motor-vehicle,1/31/2018 0:55,1/31/2018 6:55,2018-01-31,2828 N ZUNI ST,...,-105.015451,39.757627,1,113,highland,1,0,2018,1/31/2018,2018A
10,2.018724e+08,2.020000e+14,3572,0,drug-methampetamine-possess,drug-alcohol,1/30/2018 20:04,NaN,2018-01-30,E EVANS AVE / S GRAPE ST,...,-104.925197,39.678463,3,323,goldsmith,1,0,2018,1/30/2018,2018A


In [95]:
color_list = ['red','purple','orange', 
              'black','purple','dodgerblue','teal',
              'mediumblue','darkviolet',
              'purple', 'white','gray','greenyellow']
df['OFFENSE_CATEGORY_ID'].unique()


array(['all-other-crimes', 'larceny', 'theft-from-motor-vehicle',
       'drug-alcohol', 'auto-theft', 'white-collar-crime', 'burglary',
       'public-disorder', 'aggravated-assault',
       'other-crimes-against-persons', 'robbery', 'sexual-assault',
       'murder', 'arson'], dtype=object)

# Main Code to Run

In [209]:
year_list= ['2013A','2015A','2017A','2018A']
color_list = ['purple','dodgerblue']
crime_list = ['larceny','drug-alcohol']

for year in year_list:
    plot_title = f"Crime in the Year {year}   green= dispensaries"
    map_options = GMapOptions(lat=39.738711, lng=-104.989411, map_type="roadmap", zoom=12)
    plot = GMapPlot(
        x_range= Range1d(), y_range= Range1d(), map_options= map_options
    )
    plot.api_key = toby_api_key
    
    for i in range(len(crime_list)):
        lat, lng = get_crime_type_locations(crime_list[i], year)
        map_plot(plot, lat, lng, color_list[i], 3)
        plot_title += f"   {color_list[i]}= {crime_list[i]}"
    
    map_plot(plot, business_lats, business_lngs, 'green', 6)   
    plot.add_tools(WheelZoomTool())
    plot.add_tools(PanTool())
    plot.title.text = plot_title
    show(plot)

In [220]:
year_list= ['2013A','2015A','2017A','2018A']
crime_list = ['larceny','drug-alcohol']

for year in year_list:
    for crime in crime_list:
        year_crime_count = df[(df['OFFENSE_CATEGORY_ID'] == crime) & (df['year'] == year)]['year'].count()
        print(f'[{year} {crime} =  {year_crime_count}')
        

[2013A larceny =  3337
[2013A drug-alcohol =  1487
[2015A larceny =  4163
[2015A drug-alcohol =  3125
[2017A larceny =  4202
[2017A drug-alcohol =  2842
[2018A larceny =  4262
[2018A drug-alcohol =  3010


In [210]:
df.head()

,INCIDENT_ID,OFFENSE_ID,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,REPORTED_DATE,INCIDENT_ADDRESS,...,GEO_LON,GEO_LAT,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,IS_CRIME,IS_TRAFFIC,Year,semi_year,year
0,2.016377e+09,2.020000e+15,5213,0,weapon-unlawful-discharge-of,all-other-crimes,6/15/2016 23:31,NaN,2016-06-15,NaN,...,-104.809881,39.773188,5,521,montbello,1,0,2016,6/15/2016,2016A
1,2.018600e+10,2.020000e+16,2399,0,theft-other,larceny,10/11/2017 12:30,10/11/2017 16:55,2018-01-29,NaN,...,-104.781434,39.785649,5,522,gateway-green-valley-ranch,1,0,2018,1/29/2018,2018A
2,2.016600e+10,2.020000e+16,2305,0,theft-items-from-vehicle,theft-from-motor-vehicle,3/4/2016 20:00,4/25/2016 8:00,2016-04-26,2932 S JOSEPHINE ST,...,-104.957381,39.663490,3,314,wellshire,1,0,2016,4/26/2016,2016A
3,2.018723e+08,2.020000e+14,2399,0,theft-other,larceny,1/30/2018 19:20,NaN,2018-01-30,705 S COLORADO BLVD,...,-104.941440,39.702698,3,312,belcaro,1,0,2018,1/30/2018,2018A
4,2.017411e+09,2.020000e+15,2303,0,theft-shoplift,larceny,6/22/2017 20:53,NaN,2017-06-23,2810 E 1ST AVE,...,-104.955370,39.717107,3,311,cherry-creek,1,0,2017,6/23/2017,2017A


## Mapping Experiment

In [198]:
year_list= ['2018A']
color_list = ['black','dodgerblue','orange', 
              'red','purple','aqua','teal',
              'mediumblue','darkviolet',
              'purple', 'white','gray','greenyellow']
crime_list = ['auto-theft','drug-alcohol','public-disorder','larceny','theft-from-motor-vehicle']
crime = 'auto-theft'

for year in year_list:
    map_options = GMapOptions(lat=39.738711, lng=-104.989411, map_type="roadmap", zoom=13)
    plot = GMapPlot(
        x_range= Range1d(), y_range= Range1d(), map_options= map_options
    )
    map_plot(plot, business_lats, business_lngs, 'green', 6)
    plot.api_key = toby_api_key
    
    lat, lng = get_crime_type_locations(crime, year)
    map_plot(plot, lat, lng, color_list[0], 3)
    
    plot.add_tools(WheelZoomTool())
    plot.add_tools(PanTool())
    plot.title.text = crime_list
    show(plot)

In [25]:
lat_2018A = df[df['year'] == '2018A']['GEO_LAT'].dropna()
lat_2018A = lat_2018A[lat_2018A > 38].copy()
lat_2018A = lat_2018A[lat_2018A < 41].copy()

lng_2018A = df[df['year'] == '2018A']['GEO_LON'].dropna()
lng_2018A = lng_2018A[lng_2018A_place > -105.5].copy()
lng_2018A = lng_2018A[lng_2018A < -104].copy()

lat_2017A = df[df['year'] == '2017A']['GEO_LAT'].dropna()
lat_2017A = lat_2017A[lat_2017A > 38].copy()
lat_2017A = lat_2017A[lat_2017A < 41].copy()

lng_2017A = df[df['year'] == '2017A']['GEO_LON'].dropna()
lng_2017A = lng_2017A[lng_2017A > -105.5].copy()
lng_2017A = lng_2017A[lng_2017A < -104].copy()

lng_2018A.head()


1   -104.781434
3   -104.941440
5   -104.961928
6   -105.025543
7   -104.983794
Name: GEO_LON, dtype: float64

In [92]:
map_options = GMapOptions(lat=39.738711, lng=-104.989411, map_type="roadmap", zoom=11)
plot = GMapPlot(
    x_range= Range1d(), y_range= Range1d(), map_options=map_options
)
plot.title.text = "Hey look! It's a scatter plot on a map!"


color_blue = ['blue' for each in lat_2018A]
source = ColumnDataSource(
    data=dict(
        lat= lat_2018A,
        lon= lng_2018A,
        size= 1.5*np.ones(len(lat_2018A)),
        color = color_blue
    
    )
)
circle = Circle(x="lon", y="lat", size="size", fill_color= 'color', fill_alpha=0.5, line_color=None)
plot.add_glyph(source, circle)

color_yellow = ['yellow' for each in lat_2017A]
source2 = ColumnDataSource(
    data=dict(
        lat= lat_2017A,
        lon= lng_2017A,
        size= 1.5*np.ones(len(lat_2017A)),
        color = color_yellow
    )
)
circle2 = Circle(x="lon", y="lat", size="size", fill_color= 'color', fill_alpha=0.5, line_color=None)
plot.add_glyph(source2, circle2)

legend_thang = Legend(items=[
    LegendItem(label="2018A"),
    LegendItem(label="2017A")],
    click_policy = 'hide', location = 'top_left')

plot.add_layout(legend_thang)

plot.api_key = toby_api_key

show(plot)